# A. Physical Lecture: Minimum Variance Frontier: Constrained



## A.1 Set-up

An investor considers an investment into three already well diversified exchange traded funds. Asset 1 has an expected return of 3%, a volatility of 4% and pairwise correlations to asset 2 and 3 of 10% and 20%, respectively. Asset 2 has an expected return of 6%, a volatility of 13% and a pairwise correlation with asset 3 of 70%.  Asset 3 has an expected return of 8% and a volatility of 16%.

The investor wants to inspect a subset of minimum variance portfolios. The subset of interest must not contain any short sale positions and none of the positions shall make up more than 90% of the investment. 

## A.2  Code Requirements
 
We need numpy for algebra, scipy.optimize for the numerical optimization and matplotlib for plotting.

In [111]:
# install needed packages
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

In [112]:
# Code set up:
r_1 = 0.03
vol_1 = 0.04
p1_2 = 0.1
p1_3 = 0.2
r_2 = 0.06
vol_2 = 0.13
p2_3 = 0.7
r_3 = 0.08
vol_3 = 0.16
# no short selling, minimum variance frontier

## A.3 Investment Opportunity Set
= mu and sigma and risk-free rate (Markovitz)

In [113]:
# mu being a column vector
mu = np.array([[r_1],[r_2],[r_3]])
print(mu)

cov_mat = np.matrix([[vol_1*vol_1,vol_1*vol_2*p1_2,vol_1*vol_3*p1_3]
                    ,[vol_2*vol_1*p1_2,vol_2*vol_2,vol_2*vol_3*p2_3]
                    ,[vol_1*vol_3*p1_3,vol_2*vol_3*p2_3,vol_3*vol_3]]
                    )
print(cov_mat)


[[0.03]
 [0.06]
 [0.08]]
[[0.0016  0.00052 0.00128]
 [0.00052 0.0169  0.01456]
 [0.00128 0.01456 0.0256 ]]


## A.4 Objective Function

Now, we define the objective function that the investor wants to opimize. In our application that is the portfolio variance (as a function of the portfolio weights). The objective function needs to return a scalar value.  

In [114]:
def portfolio_variance(w):
    """
    :param w: column vector of weights
    :return: portfolio variance: 1x1: not as a vector, but as skalar
    """
    return (w.T * cov_mat * w).item()
                   
    

## A.5 Equality Constraints of Optimization Problem

Now, we set-up functions for the equality constraints of the optimization problem. 

We re-write equality constraints of the problem as

$$
f(x) = 0
$$  



### A.5.1 Full Investment Constraint

We start with the full investment constraint: All money must be invested

In [115]:
def full_invest_contraint(w):
    """
    :param w: column vector of weights
    :return:  w.T * e = 1 : weights need to add up to 1
    """
    return np.sum(w) - 1

    

### A.5.2 Target Return Constraint

We continue with the target portfolio return constraint. mu_tagert = w.T * mu

Notice: the next line is a quick and dirty way of coding. It is dirty because it uses a "global variable", here mu_target, which as of now is not defined. Yet, Python accepts that as it only requires to know that global quantity once it executes the function for the first time

In [116]:
def target_constr(w):
    # doesn't work: dimensions don't fit
    return (w * mu.T).item() - mu_target


## A.6 Starting Value for Parameter Search

We set the starting value for the three dimensional parameter space to 

$$
w_0 = [0.4, 0, 0.6].
$$

Important: Ensure the starting values of an optimization problem fullfill the constraints.

In [117]:
w0 = np.array([[0.4],[0],[0.6]])
# w0 = [0.4,0,0.6]

## A.7 Set Parameter Bounds for Individual Parameters

We set the bounds, which coincide with constraints on single parameters, here

$$
0 \leq w_j \leq 0.9
$$ 

In [118]:
# j from 1 to 3: 3 assets: every asset could have it own contrstraint
bounds = [
    (0,0.9),
    (0,0.9),
    (0,0.9),
]


## A.8 Set Constraints on a Combination of Parameters

We set constraints, which coincide with functions on several parameters. 

Here, that is the **full investment constraint** and the **target return constraint**. 

The normed structure follows https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

In our application, we say to work with 'equality constraints' that are parsed as 'functions'

In [119]:
constraints = [
    {'type': 'eq','fun': target_constr},
    {'type': 'eq','fun': full_invest_contraint}
               ]


## A.9 Setting Target (Expected) Returns for which we compute the MV-Frontier

Now, we define a set of target return for which we want to find the minimum variance portfolio that fullfills the constraints. 

We set range of target expected returns **(lower bound, increment, upper bound)** for which we seek to find the minimum variance portfolio. 

Notice, not all target returns can be reached due to the specific constraints. For example, in our current example, an expected portfolio return of 10% is not possible since shorting is not allowed and the highest single asset expected return is 8%. We can easily see that the achievable **lower bound** is 

$$
3.3\%=0.9 * 3\% + 0.1 * 6\%
$$

and the achievable **upper bound** is

$$
7.8\% = 0.9 * 8\% +0.1 * 6\%
$$

This allows us to set natural lower and upper bounds. As to the increments, we work with 10 basis points, i.e.

$$
incr = 0.1\%
$$
 

In [120]:
mu_lb = 0.033
mu_ub = 0.078
incr = 0.001

## A.10 Computing the Constrained Minimum-Variance Frontier

We now loop through all traget returns to find the resulting constrained minimum variance portfolio. We store the portfolio weights in a variable 

$$
\text{w_cmv},
$$

the expected returns of the portfolios in

$$
\text{mu_cmv},
$$

and their volatility in

$$
\text{sigma_cmv}
$$

In [130]:
# iterations through for loop
n = int(1 + (mu_ub - mu_lb)/incr)

# zero vector of weights nx3
w_cmv = np.zeros((n,3))

# zero vector of mu nx1
mu_cmv = np.zeros((n,1))

# zero vector of sigma nx1
sigma_cmv = np.zeros((n,1))

# define target return
mu_target = mu_lb

# optimazation
for i in range(0,n):
    # doesn't work and should get aout 1x1 , not 3x3: not possible from 1x3 * 3x1 matrix: ERROR
    print(w0.T * mu)
    w_opt = scipy.optimize.minimize(portfolio_variance,w0,bounds = bounds, constraints= constraints)
    w_cmv[i,:] = w_opt.x
    mu_cmv[i,0] = (w_cmv[i,:] * mu).item()
    sigma_cmv[i,0] = (w_cmv[i,:] * sigma_cmv * w_cmv[i,:].T).item() ** 0.5

    mu_target += incr

[[0.012 0.    0.018]
 [0.024 0.    0.036]
 [0.032 0.    0.048]]


ValueError: can only convert an array of size 1 to a Python scalar

## A.12 Plotting the Constrained Minimum Variance Frontier

In [ ]:
plt.figure(figsize = (8,5))

plt.scatter(sigma_cmv,mu_cmv,label = "constrained MV portfolio")

plt.legend(loc = "best")
plt.show()
